In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

DATA = "data/01-Name-Link.csv"
DATA_PATH = "data/02-Info.csv"

# 读取数据
name_link = pd.read_csv(DATA, encoding="utf-8")
name_link.head()

,name,link
0,阿米娅,https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85
1,郁金香,https://prts.wiki/w/%E9%83%81%E9%87%91%E9%A6%99
2,杜宾,https://prts.wiki/w/%E6%9D%9C%E5%AE%BE
3,末药,https://prts.wiki/w/%E6%9C%AB%E8%8D%AF
4,红豆,https://prts.wiki/w/%E7%BA%A2%E8%B1%86


In [2]:
# 将df拆解成几个列表
list_name = name_link['name'].tolist()
list_link = name_link['link'].tolist()

In [3]:
def _parse_webpage(link):
    r = requests.get(link)
    r.encoding = "utf-8"
    html = r.text
    soup = BeautifulSoup(html, "html.parser")
    return soup
# region tables
def _get_info_table_1(table):
    """获得信息“分支” “描述” """
    # 找到第二个tr，获取其中第一个td中的文本，存为“分支”；第二个td中的所有文本（可能会有部分文本在span内），存为“描述”
    tr = table.find_all("tr")[1]
    td = tr.find_all("td")
    archetype = td[0].text.split("\n")[0]
    traits = td[1].text.split("\n")[0]
    return archetype, traits

def _get_info_table_2(table):
    """获得信息“再部署时间” “部署费用” “阻挡数” “攻击间隔” """
    # 获取4个td中的文本，存为所需信息
    td = table.find_all("td")
    redeploy_time = td[0].text.split("\n")[0]
    dp_cost = td[1].text.split("\n")[0]
    block = td[2].text.split("\n")[0]
    attack_interval = td[3].text.split("\n")[0]
    return redeploy_time, dp_cost, block, attack_interval

def _get_info_table_3(table):
    """
    获得信息“生命上限” “攻击” “防御” “法术抗性”
    其中每一类信息有5种值，我们以后缀表示：_0_1, _0_max, _1_max, _2_max, trust_extra_status
    """
    # 我们用列表来储存这些信息
    max_hp = []
    atk = []
    def_ = []
    res = []
    all_info = [max_hp, atk, def_, res]
    # 找到所有tr，跳过第一个，从剩下的4个tr中获取信息
    # 剩下的每个tr里的td储存了信息，获取它们（如果为空，就用0代替）
    tr = table.find_all("tr")
    for i in range(1, 5):
        td = tr[i].find_all("td")
        for j in range(5):
            text = td[j].text.split("\n")[0]
            if text == "":
                text = "0"
            all_info[i-1].append(text)
    return max_hp, atk, def_, res

def _get_info_table_4(table):
    """获取信息“攻击范围”，返回一个列表"""
    range_amount = []
    # 第二个tr中有3或者2个td，每个td中<use>的数量就是攻击范围
    tr = table.find_all("tr")[1]
    tds = tr.find_all("td")
    for td in tds:
        range_amount.append(len(td.find_all("use")))
    return range_amount

# endregion
def _get_info_single_page(link, verbose=False):
    """获取单个页面的信息：分支、描述、再部署时间、部署费用、阻挡数、攻击间隔、"""
    _start_time_0 = time.time()
    
    page = _parse_webpage(link)
    
    _end_time_0 = time.time()
    if verbose:
        print(f"soup time: {_end_time_0 - _start_time_0}")

    _start_time_1 = time.time()
    table1 = page.find_all("table", class_="wikitable logo")[0]
    table2 = page.find_all("table", class_="wikitable logo char-extra-attr-table")[0]
    table3 = page.find_all("table", class_="wikitable logo char-base-attr-table")[0]
    table4 = page.find_all("table", class_="wikitable nomobile logo")[0] # 攻击范围  

    archetype, traits = _get_info_table_1(table1)
    redeploy_time, dp_cost, block, attack_interval = _get_info_table_2(table2)
    max_hp, atk, def_, res = _get_info_table_3(table3)
    range_amount = _get_info_table_4(table4)

    _end_time_1 = time.time()
    if verbose:
        print(f"table time: {_end_time_1 - _start_time_1}")
    
    return archetype, traits, redeploy_time, dp_cost, block, attack_interval, max_hp, atk, def_, res, range_amount
def get_info(df):
    """遍历所有数据，获取信息，存入info中（replace=True）"""
    all_info = ["分支", "描述", 
                "再部署时间", "部署费用", "阻挡数", "攻击间隔",
                "生命上限", "攻击", "防御", "法术抗性",
                "攻击范围",]
    info = pd.DataFrame(columns=all_info)
    for i in tqdm(range(len(df))):
        print(f"current operator: {df.iloc[[i]]['name'].values[0].strip()}")
        row = df.iloc[[i]]
        link = row["link"].values[0].strip()
        archetype, traits, redeploy_time, dp_cost, block, attack_interval, max_hp, atk, def_, res, range_amount = _get_info_single_page(link)
        info.loc[i, "分支"] = archetype
        info.loc[i, "描述"] = traits
        info.loc[i, "再部署时间"] = redeploy_time
        info.loc[i, "部署费用"] = dp_cost
        info.loc[i, "阻挡数"] = block
        info.loc[i, "攻击间隔"] = attack_interval
        info.loc[i, "生命上限"] = max_hp
        info.loc[i, "攻击"] = atk
        info.loc[i, "防御"] = def_
        info.loc[i, "法术抗性"] = res
        info.loc[i, "攻击范围"] = range_amount

    return info

# table2 = _get_info_single_page("https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85")
# _get_info_single_page("https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85", verbose=True)

In [4]:
# # 用随机5个数据测试
# np.random.seed(42)
# test_index = np.random.randint(0, len(name_link), 5)

# # 用这5个数据测试
# test_data = name_link.iloc[test_index]
# print(test_data.iloc[:, 0])

# get_info(test_data)

In [5]:
info = get_info(name_link)

  0%|          | 0/293 [00:00<?, ?it/s]

current operator: 阿米娅


  0%|          | 1/293 [00:01<07:39,  1.57s/it]

current operator: 郁金香


  1%|          | 2/293 [00:07<19:36,  4.04s/it]

...

 99%|█████████▉| 291/293 [12:38<00:06,  3.16s/it]

current operator: 铎铃


100%|█████████▉| 292/293 [12:39<00:02,  2.55s/it]

current operator: 仇白


100%|██████████| 293/293 [12:41<00:00,  2.60s/it]


In [10]:
# 在info的开头加入name_link中的name列
info = pd.concat([name_link.iloc[:,0], info], axis=1)

In [11]:
# 将info存入文件
info.to_csv(DATA_PATH, index=False)